In [1]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from requests import get
from pytube import YouTube
import os
from youtube_dl import YoutubeDL

In [2]:
spotify = spotipy.Spotify(client_credentials_manager=SpotifyClientCredentials(
    client_id = "9d72a19e7341404da60fe2b246407fbe",
    client_secret = "b38db97775fc41519e4b533818f1e04b" 
))

In [3]:
def playlist(playlist_id):
    tracks_response = spotify.playlist_tracks(playlist_id, market = "IN")
    tracks = tracks_response["items"]
    while tracks_response["next"]:
        tracks_response = spotify.next(tracks_response)
        tracks.extend(tracks_response["items"])

    return tracks

In [4]:
YDL_OPTIONS = {'format': 'bestaudio', 'noplaylist':'True'}

def search(arg):
    arg = arg.replace(" ","+")
    arg = arg + "+song"
    with YoutubeDL(YDL_OPTIONS) as ydl:
        try:
            get(arg) 
        except:
            video = ydl.extract_info(f"ytsearch:{arg}", download=False)['entries'][0]
        else:
            video = ydl.extract_info(arg, download=False)

    return video

In [5]:
gen = {"rock" : "3qu74M0PqlkSV76f98aqTd",
        "hiphop" : "37i9dQZF1DX48TTZL62Yht"}

In [6]:
name = []
song_id = []
genre = []
dic = {
    'name' : name,
    'id' : song_id,
    "label" : genre
}

In [7]:
for j,k in zip(gen.values(), gen.keys()):
    result = playlist(j)
    for i in range(50):
        n = result[i]['track']['name']
        s_id = search(n)['id']
        name.append(n)
        song_id.append(s_id)
        genre.append(k)

[download] Downloading playlist: Kids+In+The+Dark+song
[youtube:search] query "Kids+In+The+Dark+song": Downloading page 1
[youtube:search] playlist Kids+In+The+Dark+song: Downloading 1 videos
[download] Downloading video 1 of 1
[youtube] SaIegdJxTsE: Downloading webpage
[youtube] Downloading just video SaIegdJxTsE because of --no-playlist
[download] Finished downloading playlist: Kids+In+The+Dark+song
[download] Downloading playlist: Shots+song
[youtube:search] query "Shots+song": Downloading page 1
[youtube:search] playlist Shots+song: Downloading 1 videos
[download] Downloading video 1 of 1
[youtube] eWjGh61Gacg: Downloading webpage
[youtube] Downloading just video eWjGh61Gacg because of --no-playlist
[download] Finished downloading playlist: Shots+song
[download] Downloading playlist: Lampshades+on+Fire+song
[youtube:search] query "Lampshades+on+Fire+song": Downloading page 1
[youtube:search] playlist Lampshades+on+Fire+song: Downloading 1 videos
[download] Downloading video 1 of 1


In [8]:
import pandas as pd
df = pd.DataFrame(dic)
df.to_csv("songs.csv",header = True, index=False)

In [10]:
for id, j in zip(df['id'], df['label']):
    yt = YouTube(f'https://www.youtube.com/watch?v={id}')
    video = yt.streams.filter(only_audio=True).first()
    out_file = video.download(output_path=f"song\{j}.")
    base, ext = os.path.splitext(out_file)
    new_file = base + '.mp3'
    os.rename(out_file, new_file)